In [19]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
%load_ext rpy2.ipython 

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Aim

The aim of this notebook is to filter mobileOG-db to only keep MGEs relevant to environmental ARGs.



## Time to filter the DB

The first step is to go to `https://mobileogdb.flsi.cloud.vt.edu/entries/database_download` and download `Manually Curated Data` to dir `files`.

First We need to unzip the files and add some metadata using a script adapted from this GutHub repo
`https://github.com/clb21565/mobileOG-db/blob/main/scripts/getElementClassifications.R`. 

In [ ]:
%%R -o out

library(tidyverse)
library(data.table)

# Code adapted from https://github.com/clb21565/mobileOG-db/blob/main/scripts/getElementClassifications.R

`%notin%` <- Negate(`%in%`)

mobileOGs=fread("files/mobileOG-db-beatrix-1.6-MC.csv")
mobileOGs_classes=mobileOGs[,c("mobileOG Entry Name","Major mobileOG Category","Minor mobileOG Categories","ACLAME","immedb","GPD",
            "pVOG","ICE","AICE","CIME","IME","ISFinder","PlasmidRefSeq","COMPASS")]

TE=mobileOGs_classes%>%subset(ISFinder!=0)
TE$MGE_Class="Insertion Sequence (IS)"
IGE=mobileOGs_classes%>%subset(AICE!=0|ICE!=0|CIME!=0|immedb!=0|IME!=0)%>%subset(ISFinder==0)
IGE$MGE_Class="Integrative Element (IGE)"
Ph=mobileOGs_classes%>%subset(GPD!=0|pVOG!=0)
Ph$MGE_Class="Phage"
plasmidic=mobileOGs_classes%>%subset(COMPASS!=0|PlasmidRefSeq!=0)%>%
  filter(`mobileOG Entry Name`%notin%c(IGE$`mobileOG Entry Name`))%>%
  subset(ISFinder==0)
plasmidic$MGE_Class="Plasmid"
CE=mobileOGs_classes%>%subset(`Major mobileOG Category`=="transfer"&
                                grepl("conjugati",`Minor mobileOG Categories`))
CE$MGE_Class="Conjugative Element (CE)"

merged=rbind(TE,IGE,Ph,plasmidic,CE)

ACLAME=mobileOGs_classes%>%filter(`mobileOG Entry Name`%notin%merged$`mobileOG Entry Name`)

ACLAME$MGE_Class="Only detected in ACLAME"

merged=rbind(merged,ACLAME)



merged[,c("mobileOG Entry Name","MGE_Class")]%>%
  group_by(`mobileOG Entry Name`)%>%
       mutate(MGE_Class = paste((MGE_Class), collapse=","))%>%ungroup()

length(unique(merged$`mobileOG Entry Name`))



out=merged%>%group_by(`mobileOG Entry Name`) %>%
  dplyr::summarize(MGE_Class = paste(unique(MGE_Class),collapse=','))%>%
  ungroup()

Here I will add the additional metadata to the metadata file.

In [21]:
# Metadata file
df_mobileog_mc = pd.read_csv("files/mobileOG-db-beatrix-1.6-MC.csv")

# Add metadata to the manually curated file
df_mobileog_mc = pd.merge(df_mobileog_mc, out, on = "mobileOG Entry Name", how = "left")

Hera are all categories in the mobileOG-db metadata file

In [22]:
for item in df_mobileog_mc.MGE_Class.unique():
    print(item)

Plasmid
Phage
Only detected in ACLAME
Phage,Plasmid
Integrative Element (IGE)
Integrative Element (IGE),Phage
Insertion Sequence (IS)
Insertion Sequence (IS),Phage
Phage,Conjugative Element (CE)
Plasmid,Conjugative Element (CE)
Conjugative Element (CE)
Integrative Element (IGE),Conjugative Element (CE)
Integrative Element (IGE),Phage,Conjugative Element (CE)


First I remove everything that contains Phage in the class, and print the remaining classes.

In [23]:
df_mobileog_mc_f = df_mobileog_mc[~df_mobileog_mc["MGE_Class"].str.contains("Phage", regex=True)]

for item in df_mobileog_mc_f.MGE_Class.unique():
    print(item)

Plasmid
Only detected in ACLAME
Integrative Element (IGE)
Insertion Sequence (IS)
Plasmid,Conjugative Element (CE)
Conjugative Element (CE)
Integrative Element (IGE),Conjugative Element (CE)


## Filter on Conjugative elements 

In [24]:
# Groups related to conjugative elements
keep_CE = ["Integrative Element (IGE),Conjugative Element (CE)", 
           "Conjugative Element (CE)", 
           "Plasmid,Conjugative Element (CE)"]

df_CE = df_mobileog_mc_f[df_mobileog_mc_f.MGE_Class.isin(keep_CE)]
df_DE_major_Class = df_CE["Major mobileOG Category"].unique()
df_DE_minor_Class = df_CE["Minor mobileOG Categories"].unique()

Bellow are all minor categories in this class. I kept them all.

In [25]:
print(f"Major Categories in this class: {df_DE_major_Class} \n")
print(f"Minor Categories in this class: {df_DE_minor_Class}")

Major Categories in this class: ['transfer'] 

Minor Categories in this class: ['conjugation' 'conjugation,relaxosome' 'conjugation,dispensible'
 'conjugation,regulation' 'conjugation,competence'
 'regulation,conjugation' 'conjugation,replication/recombination/repair'
 'conjugation,initiation']


## Filter IS elements 

In [26]:
# Here I will only keep the genes from ISfinder and remove phage
df_IS = df_mobileog_mc_f[df_mobileog_mc_f.ISFinder != 0]
df_IS = df_IS[df_IS["Major mobileOG Category"] != "phage"]

df_IS_major_Class = df_IS["Major mobileOG Category"].unique()
df_IS_minor_Class = df_IS["Minor mobileOG Categories"].unique()

I only kept genes from the ISfinder datable. Below are minor categories within this class.

In [27]:
print(f"Major Categories in this class: {df_IS_major_Class} \n")
print(f"Minor Categories in this class: {df_IS_minor_Class}")

Major Categories in this class: ['integration/excision' 'replication/recombination/repair'
 'stability/transfer/defense'] 

Minor Categories in this class: [nan 'replication/recombination/repair' 'crispr,spacer acquisition'
 'invertase,pilin variation']


## Only detected in ACLAME (an MGE database)

In [28]:
df_OIA = df_mobileog_mc_f[df_mobileog_mc_f.MGE_Class == "Only detected in ACLAME"]
df_OIA = df_OIA[df_OIA["Major mobileOG Category"] != "phage"]

df_OIA_major_Class = df_OIA["Major mobileOG Category"].unique()
df_OIA_minor_Class = df_OIA["Minor mobileOG Categories"].unique()

# Keep only gene involved in conjugation
df_OIA = df_OIA[df_OIA["Minor mobileOG Categories"] == "conjugation"]

This is a strange category which contains a lot of different major and minor categories. I choose to only keep the genes related to conjugation. I have printed all categories in the class below. 

In [29]:
print(f"Major Categories in this class: {df_OIA_major_Class} \n")
print(f"Minor Categories in this class: {df_OIA_minor_Class}")

Major Categories in this class: ['replication/recombination/repair' 'integration/excision'
 'stability/transfer/defense' 'transfer'] 

Minor Categories in this class: [nan 'replication/recombination/repair' 'regulation' 'partitioning'
 'phage' 'transfer' 'initiation,copy number control,plasmid' 'conjugation'
 'transfer,partitioning' 'shufflon' 'CRISPR' 'replication,resolvase'
 'introns' 'crispr' 'crispr,spacer acquisition' 'plasmid' 'competence'
 'stability/transfer/defense' 'replication'
 'crispr,nucleic acid degradation']


## Plasmids

In [30]:
df_plasmids = df_mobileog_mc_f[df_mobileog_mc_f.MGE_Class == "Plasmid"]

df_pl_major_Class = df_plasmids["Major mobileOG Category"].unique()
df_pl_minor_Class = df_plasmids["Minor mobileOG Categories"].unique()

# Only keep genes involved in conjugation and transfer
df_plasmids = df_plasmids[df_plasmids["Minor mobileOG Categories"] == "transfer,conjugation"]

I choose to only keep the genes related to `transfer,conjugation`. I have printed all categories in the class below. 

In [31]:
print(f"Major Categories in this class: {df_pl_major_Class} \n")
print(f"Minor Categories in this class: {df_pl_minor_Class}")

Major Categories in this class: ['integration/excision' 'phage' 'replication/recombination/repair'
 'stability/transfer/defense' 'transfer'] 

Minor Categories in this class: [nan 'replication/recombination/repair' 'stability/transfer/defense'
 'lysis/lysogeny,regulation' 'replication' 'infection,regulation'
 'regulation' 'lysis/lysogeny' 'partitioning' 'structural'
 'integration/excision' 'initiation,copy number control,plasmid' 'plasmid'
 'regulation,transfer,replication'
 'partitioning,initiation,copy number control,plasmid'
 'transfer,conjugation' 'replication,resolvase' 'CRISPR'
 'regulation,lysis/lysogeny' 'replication,packaging' 'lysis'
 'integration/excision,inversion' 'transfer,receptor' 'competence'
 'regulation,plasmid' 'transfer' 'transfer,partitioning' 'crispr'
 'relaxosome' 'shufflon' 'regulation,stability' 'linear chromosomes'
 'replication,inversion' 'infection,lysis/lysogeny' 'phage' 'introns'
 'splicing,transfer' 'replication/recombination/repair,initiation'
 'crispr,

# Integrons

In [32]:
# Keep only if we have a gene name
df_int = df_mobileog_mc_f[~df_mobileog_mc_f.Name.isna()]

# Only keep int genes
df_int = df_int[ df_int.Name.str.contains("int")]

# Keep these manual annotations
list_annotation_keep = ['Transposase required for excision and integration mobile elements',
                        'Integrase',
                        'Integrase (Recombinase)',
                        'Tyrosine Integrase Recombinase',
                        'Putative Integrase', 'serine integrase',
                        'DNA integration/recombination/inversion protein']

df_int = df_int[df_int["Manual Annotation"].isin(list_annotation_keep)]
df_int = df_int[df_int["Major mobileOG Category"] != "phage"]

This was slightly more complicated. I first looked for genes with the name ìnt*` I then looked through the manual annotations and kept these ones:
```
['Transposase required for excision and integration mobile elements',
'Integrase',
'Integrase (Recombinase)',
'Tyrosine Integrase Recombinase',
'Putative Integrase', 'serine integrase',
'DNA integration/recombination/inversion protein']
```

Finally, I removed categories related to phage. 

## Combine all categories to a final db.

In [33]:
# Define Type
df_conjugation = pd.concat([df_OIA, df_plasmids, df_CE])
df_conjugation["Type"] = "Conjugation"
df_int["Type"] = "Integrases"
df_IS["Type"] = "IS elements"

# The intI genes were identified in a different way than the other categories. 
# There might therefore be some overlap between it and the other categories
# Here I create a list of the int gene identifiers and remove them from the other categories
df_int_list = df_int["mobileOG Entry Name"].unique()
df_conjugation = df_conjugation[~df_conjugation["mobileOG Entry Name"].isin(df_int_list)]
df_IS = df_IS[~df_IS["mobileOG Entry Name"].isin(df_int_list)]
# Combine all categories
df_final = pd.concat([df_conjugation, df_int, df_IS])

# Remove everything related to phage (again)
df_final = df_final[~(df_final["Manual Annotation"].str.contains("phage")) |
                    (df_final["Minor mobileOG Categories"].str.contains("phage"))]

# Number of genes in each category

In [34]:
df_final.Type.value_counts()

Type
Conjugation    2628
IS elements     311
Integrases      121
Name: count, dtype: int64

In [35]:
#df_final.to_csv("files/mobileOG-db-beatrix-1.6-MC_filtered.csv", sep = "\t", index=False)

In [36]:
# Create a list to store SeqRecord objects
records = []

# Iterate through the DataFrame rows
for index, row in df_final.iterrows():
    # Create a SeqRecord object for each row
    record = SeqRecord(
        Seq(row['Amino Acid Sequence']),
        id=row['mobileOG fasta Header'],
        description=""
    )
    records.append(record)

# Write the records to a FASTA file
output_file = "files/mobileOG-db-beatrix-1.6-MC_filtered.fasta"
#SeqIO.write(records, output_file, "fasta")

3060